In [7]:
import ee
import geemap
ee.Authenticate()
ee.Initialize()


import pandas as pd
import geopandas as gpd
import numpy as np 
from ipyleaflet import CircleMarker
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
from shapely import wkt  # For converting text to geometry

pd.options.display.float_format = '{:.3f}'.format

In [8]:
ref_map = pd.read_csv('/home/sepal-user/COCAFORI/CMR_2020_LC_types_ceo.csv')

In [9]:
ref_map_gdf = gpd.GeoDataFrame(ref_map)
# Convert 'geometry' column from string to actual geometry objects
ref_map_gdf['geometry'] = ref_map_gdf['geometry'].apply(wkt.loads)
# Set 'geometry' as the active geometry column
ref_map_gdf = ref_map_gdf.set_geometry('geometry')

In [10]:
ref_map_gdf['lon'] = ref_map_gdf.geometry.x
ref_map_gdf['lat'] = ref_map_gdf.geometry.y

In [11]:
ref_map_gdf.head()

,geometry,FNF_2020_Ref,LC_2020_Code_Map,LC_2020_Code_Ref,collection,FNF_2020_Map,index,plotid,sampleid,lon,lat
0,POINT (14.29857 7.77672),1,11,12,1,1,0,4,4,14.299,7.777
1,POINT (13.15682 9.44489),0,30,30,1,0,1,2,2,13.157,9.445
2,POINT (13.75959 8.64718),0,30,30,1,0,2,6,6,13.760,8.647
3,POINT (14.74055 11.1445),0,30,30,1,0,3,7,7,14.741,11.144
4,POINT (13.10112 4.40264),1,11,11,1,1,4,8,8,13.101,4.403


In [12]:
# Define conditions and corresponding values
conditions = [
    (ref_map_gdf['LC_2020_Code_Map'].between(10, 19)), 
    (ref_map_gdf['LC_2020_Code_Map'].between(20, 29)),  
    (ref_map_gdf['LC_2020_Code_Map'].between(30, 39)),  
    (ref_map_gdf['LC_2020_Code_Map'].between(40, 49)),  
    (ref_map_gdf['LC_2020_Code_Map'].between(50, 59)),  
    (ref_map_gdf['LC_2020_Code_Map'].between(60, 69)),  
]

values = [10, 20, 30, 40, 50, 60]

# Assign values based on conditions
ref_map_gdf['IPCC_2020_Code_Map'] = np.select(conditions, values, default=np.nan)  # Default to NaN if no condition is met

In [14]:
# Define conditions and corresponding values
conditions = [
    (ref_map_gdf['LC_2020_Code_Ref'].between(10, 19)), 
    (ref_map_gdf['LC_2020_Code_Ref'].between(20, 29)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(30, 39)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(40, 49)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(50, 59)),  
    (ref_map_gdf['LC_2020_Code_Ref'].between(60, 69)),  
]

values = [10, 20, 30, 40, 50, 60]

# Assign values based on conditions
ref_map_gdf['IPCC_2020_Code_Ref'] = np.select(conditions, values, default=np.nan)  # Default to NaN if no condition is met

In [16]:
ref_map_gdf['IPCC_agree'] = (ref_map_gdf['IPCC_2020_Code_Map'] == ref_map_gdf['IPCC_2020_Code_Ref']).astype(int)
ref_map_gdf['LC_agree'] = (ref_map_gdf['LC_2020_Code_Map'] == ref_map_gdf['LC_2020_Code_Ref']).astype(int)
ref_map_gdf['FNF_agree'] = (ref_map_gdf['FNF_2020_Map'] == ref_map_gdf['FNF_2020_Ref']).astype(int)

In [17]:
ref_map_gdf.head()

,geometry,FNF_2020_Ref,LC_2020_Code_Map,LC_2020_Code_Ref,collection,FNF_2020_Map,index,plotid,sampleid,lon,lat,IPCC_2020_Code_Map,IPCC_2020_Code_Ref,IPCC_agree,LC_agree,FNF_agree
0,POINT (14.29857 7.77672),1,11,12,1,1,0,4,4,14.299,7.777,10.000,10.000,1,0,1
1,POINT (13.15682 9.44489),0,30,30,1,0,1,2,2,13.157,9.445,30.000,30.000,1,1,1
2,POINT (13.75959 8.64718),0,30,30,1,0,2,6,6,13.760,8.647,30.000,30.000,1,1,1
3,POINT (14.74055 11.1445),0,30,30,1,0,3,7,7,14.741,11.144,30.000,30.000,1,1,1
4,POINT (13.10112 4.40264),1,11,11,1,1,4,8,8,13.101,4.403,10.000,10.000,1,1,1


In [18]:
ref_map_gdf['LC_agree'].value_counts(dropna=False)

LC_agree
1    1507
0     910
Name: count, dtype: int64

In [19]:
ref_map_gdf['IPCC_agree'].value_counts(dropna=False)

IPCC_agree
1    1891
0     526
Name: count, dtype: int64

In [20]:
ref_map_gdf = ref_map_gdf.set_crs("EPSG:4326", inplace=False)

In [21]:
# Initialize Earth Engine
ee.Initialize()

# Create the map
Map = geemap.Map(center=[0, 15], zoom=4)

# Load the GEE landcover image
landcover = ee.Image("projects/ee-cocoacmr/assets/outputs/CMR_LC_2020")  # replace with your asset

# Landcover visualization
landcover_vis = {
    'min': 11,
    'max': 60,
    'palette': [
        '#157A46', '#83C16F', '#9E8826', '#9e40b8', '#998056', '#000000', '#000000', '#000000',
        '#000000', '#000000', '#A52A2A', '#AEDF18', '#000000', '#000000', '#000000', '#000000',
        '#000000', '#000000', '#000000', '#C1D3AE', '#000000', '#000000', '#000000', '#000000',
        '#000000', '#000000', '#000000', '#000000', '#000000', '#544C90', '#1B2FEA', '#000000',
        '#000000', '#000000', '#000000', '#000000', '#000000', '#000000', '#784a50', '#000000',
        '#000000', '#000000', '#000000', '#000000', '#000000', '#000000', '#000000', '#000000',
        '#d9d2d0'
    ]
}

# Add landcover to map
Map.addLayer(landcover, landcover_vis, "Landcover")

# --- Load your point layer ---
# Example: ref_map_gdf = gpd.read_file("your_file.geojson")

# Convert LC_agree to colors
# Map LC_agree values to color strings
ref_map_gdf['color'] = ref_map_gdf['LC_agree'].map({0: '#000000', 1: '#544C90'})

Map.add_gdf(ref_map_gdf, layer_name='Ref Points', info_mode='on_hover', style={
    'color': 'color',
    'radius': 5,
    'fillOpacity': 0.8
})


# Show the map
#Map


##### add admin attributes

In [22]:
poly = "/home/sepal-user/admin/cmr_admbnda_adm1_inc_20180104.shp"
poly_shp = gpd.read_file(poly)
poly_shp.head()

,Shape_Leng,Shape_Area,ADM1_EN,ADM1_FR,ADM1_PCODE,ADM1_REF,ADM1ALT1EN,ADM1ALT2EN,ADM1ALT1FR,ADM1ALT2FR,ADM0_EN,ADM0_FR,ADM0_PCODE,date,validOn,validTo,geometry
0,15.151,5.236,Adamawa,Adamaoua,CM001,None,None,None,None,None,Cameroon,Cameroun (le),CM,2018-12-17,2019-01-04,NaT,"POLYGON ((12.28874 8.1817, 12.28379 8.16514, 1..."
1,16.171,5.602,Centre,Centre,CM002,None,None,None,None,None,Cameroon,Cameroun (le),CM,2018-12-17,2019-01-04,NaT,"POLYGON ((11.99753 6.26224, 11.99793 6.25936, ..."
2,18.826,8.957,East,Est,CM003,None,None,None,None,None,Cameroon,Cameroun (le),CM,2018-12-17,2019-01-04,NaT,"POLYGON ((14.39484 6.06262, 14.39542 6.06071, ..."
3,13.273,2.827,Far-North,Extrême-Nord,CM004,None,None,None,None,None,Cameroon,Cameroun (le),CM,2018-12-17,2019-01-04,NaT,"POLYGON ((14.53742 12.94356, 14.54344 12.93673..."
4,10.673,1.646,Littoral,Littoral,CM005,None,None,None,None,None,Cameroon,Cameroun (le),CM,2018-12-17,2019-01-04,NaT,"POLYGON ((9.84789 5.33234, 9.84934 5.32882, 9...."


In [23]:
# from the table above, identify the column name you want to associate
admin_name = 'ADM1_EN'
new_name = 'Region'

In [24]:
poly_crs = poly_shp.crs
print("Current CRS:", poly_crs)

Current CRS: EPSG:4326


In [25]:
poly_shp = poly_shp.to_crs(ref_map_gdf.crs)

In [26]:
# Reproject both GeoDataFrames to the same projected CRS (e.g., EPSG:3395 for World Mercator)
gdf = ref_map_gdf
gdf = gdf.to_crs("EPSG:3395")
poly_shp = poly_shp.to_crs("EPSG:3395")

# Step 1: Perform the spatial join for intersecting points
joined_data_intersecting = gpd.sjoin(gdf, poly_shp[[admin_name, 'geometry']], how="left", predicate="within", lsuffix='left', rsuffix='right')

# Drop the 'index_right' column if it exists, to avoid conflicts
if 'index_right' in joined_data_intersecting.columns:
    joined_data_intersecting = joined_data_intersecting.drop(columns='index_right')

# Step 2: Identify points without an intersection (NaN values in the Admin_Name column)
no_intersection_points = joined_data_intersecting[joined_data_intersecting[admin_name].isna()]

# Step 3: Perform the nearest spatial join for points without an intersection
nearest_join = gpd.sjoin_nearest(no_intersection_points.drop(columns=admin_name), poly_shp[[admin_name, 'geometry']], how="left", distance_col="distance_to_polygon")

# Drop the 'index_right' column if it exists in the nearest join result
if 'index_right' in nearest_join.columns:
    nearest_join = nearest_join.drop(columns='index_right')

# Step 4: Combine intersecting and nearest joined data into one GeoDataFrame
# Retain only the specified `admin_name` column from `poly_shp`
joined_data_combined = pd.concat([
    joined_data_intersecting.dropna(subset=[admin_name]),
    nearest_join
])

# Select only the columns from `gdf` plus `admin_name`
columns_to_keep = list(gdf.columns) + [admin_name]
joined_data = joined_data_combined[columns_to_keep]

# Display the resulting GeoDataFrame
joined_data.head()

,geometry,FNF_2020_Ref,LC_2020_Code_Map,LC_2020_Code_Ref,collection,FNF_2020_Map,index,plotid,sampleid,lon,lat,IPCC_2020_Code_Map,IPCC_2020_Code_Ref,IPCC_agree,LC_agree,FNF_agree,color,ADM1_EN
0,POINT (1591710 862592.576),1,11,12,1,1,0,4,4,14.299,7.777,10.000,10.000,1,0,1,#000000,North
1,POINT (1464610 1049187.28),0,30,30,1,0,1,2,2,13.157,9.445,30.000,30.000,1,1,1,#544C90,North
2,POINT (1531710 959855.302),0,30,30,1,0,2,6,6,13.760,8.647,30.000,30.000,1,1,1,#544C90,North
3,POINT (1640910 1240244.04),0,30,30,1,0,3,7,7,14.741,11.144,30.000,30.000,1,1,1,#544C90,Far-North
4,POINT (1458410.001 487305.28),1,11,11,1,1,4,8,8,13.101,4.403,10.000,10.000,1,1,1,#544C90,East


In [27]:
joined_data[admin_name].value_counts(dropna=False)

ADM1_EN
East          479
North         449
Adamawa       345
Centre        326
Far-North     239
South         173
North-West    117
South-West    112
Littoral       93
West           84
Name: count, dtype: int64

In [28]:
# Rename columns
column_mapping = {
   admin_name: new_name
#    'NAME_2': 'Territoire',
#    'Unnamed: 0':'Index'
}

# Use the rename() method to rename columns
joined_data.rename(columns=column_mapping, inplace=True)

In [29]:
print(joined_data.columns.tolist())

['geometry', 'FNF_2020_Ref', 'LC_2020_Code_Map', 'LC_2020_Code_Ref', 'collection', 'FNF_2020_Map', 'index', 'plotid', 'sampleid', 'lon', 'lat', 'IPCC_2020_Code_Map', 'IPCC_2020_Code_Ref', 'IPCC_agree', 'LC_agree', 'FNF_agree', 'color', 'Region']


In [30]:
joined_data = joined_data.drop(columns=['color'])

In [31]:
joined_data.to_csv('/home/sepal-user/COCAFORI/CMR_2020_LC_types_ceo_ref_val.csv',index=True)